In [1]:
!pip install -q --upgrade pip
!pip install -q tensorflow-gpu==2.2.0
!pip install -q transformers

     |████████████████████████████████| 1.5MB 2.7MB/s 
     |███▉                            | 61.3 MB 1.7 MB/s eta 0:04:22ERROR: Exception:
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/pip/_vendor/urllib3/response.py", line 425, in _error_catcher
    yield
  File "/usr/local/lib/python3.6/dist-packages/pip/_vendor/urllib3/response.py", line 507, in read
    data = self._fp.read(amt) if not fp_closed else b""
  File "/usr/local/lib/python3.6/dist-packages/pip/_vendor/cachecontrol/filewrapper.py", line 62, in read
    data = self.__fp.read(amt)
  File "/usr/lib/python3.6/http/client.py", line 459, in read
    n = self.readinto(b)
  File "/usr/lib/python3.6/http/client.py", line 503, in readinto
    n = self.fp.readinto(b)
  File "/usr/lib/python3.6/socket.py", line 586, in readinto
    return self._sock.recv_into(b)
  File "/usr/lib/python3.6/ssl.py", line 1012, in recv_into
    return self.read(nbytes, buffer)
  File "/usr/lib/python3.6/ssl.py", lin

In [2]:
!git clone https://github.com/rgrupesh/Deep_Learning.git

Cloning into 'Deep_Learning'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 57 (delta 23), reused 21 (delta 8), pack-reused 0
Unpacking objects: 100% (57/57), done.


In [0]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.python.lib.io.tf_record import TFRecordWriter

In [4]:
train=pd.read_csv("/content/Deep_Learning/data/train.csv")
train.reset_index(inplace=True)
train["sentiment"].replace({"pos":1,"neg":0}, inplace=True)

test=pd.read_csv("/content/Deep_Learning/data/test.csv")
test.reset_index(inplace=True)
test["sentiment"].replace({"pos":1,"neg":0}, inplace=True)

train.tail()

,index,text,sentiment
24995,24995,"This film is fun, if your a person who likes a...",1
24996,24996,After seeing this film I feel like I know just...,1
24997,24997,first this deserves about 5 stars due to actin...,0
24998,24998,If you like films that ramble with little plot...,0
24999,24999,"As interesting as a sheet of cardboard, this d...",0


In [5]:
test.head()

,index,text,sentiment
0,0,"My daughter liked it but I was aghast, that a ...",0
1,1,I... No words. No words can describe this. I w...,0
2,2,this film is basically a poor take on the old ...,0
3,3,"This is a terrible movie, and I'm not even sur...",0
4,4,First of all this movie is a piece of reality ...,1


In [6]:
SAMPLE_F=0.2
TRAIN_F=0.8

train_sample=train.sample(frac=SAMPLE_F, random_state=0)

train_select=train_sample.sample(frac=TRAIN_F, random_state=0)
train_csv= train_select.values

validate_select=train_sample.drop(index=train_select.index)
validate_csv= validate_select.values


test_sample=test.sample(frac=SAMPLE_F, random_state=0)
test_csv=test_sample.values

n_Tr=len(train_csv)
n_Val=len(validate_csv)
n_Te=len(test_csv)

n_Tr,n_Val,n_Te


(4000, 1000, 5000)

In [0]:
def create_example(features,label):
      tf_example = tf.train.Example(features = tf.train.Features(feature = {
        'idx': tf.train.Feature(int64_list=tf.train.Int64List(value=[features[0]])),
        'sentence': tf.train.Feature(bytes_list=tf.train.BytesList(value=[features[1].encode('utf-8')])),
        'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[label]))
    }))
      return tf_example.SerializeToString()

In [8]:
import time

def csv_to_tfRecord(csv, file_name):

  start_time = time.time()
  writer=TFRecordWriter(file_name)
  for index, row in enumerate(csv):
    features, label= row[:-1], row[-1]
    example=create_example(features, label)
    writer.write(example)
  writer.close()
  print(f"{file_name}:=====>>>{time.time() - start_time} secs")  

csv_to_tfRecord(validate_csv,"validate.tfrecord")
csv_to_tfRecord(train_csv,"train.tfrecord")
csv_to_tfRecord(test_csv,"test.tfrecord")  

validate.tfrecord:=====>>>0.06566905975341797 secs
train.tfrecord:=====>>>0.2157611846923828 secs
test.tfrecord:=====>>>0.28504490852355957 secs


In [0]:
!pip install -q transformers

In [0]:
import tensorflow as tf
from transformers import TFXLNetForSequenceClassification, glue_convert_examples_to_features, XLNetConfig
from transformers import *


In [0]:
train_ds=tf.data.TFRecordDataset("train.tfrecord")
val_ds= tf.data.TFRecordDataset("validate.tfrecord")
test_ds= tf.data.TFRecordDataset("test.tfrecord")

In [0]:
feature_spec = {
    'idx': tf.io.FixedLenFeature([], tf.int64),
    'sentence': tf.io.FixedLenFeature([], tf.string),
    'label': tf.io.FixedLenFeature([], tf.int64)
}
def parse_example(example_proto):
    return tf.io.parse_single_example(example_proto, feature_spec)

train_parse_ds=train_ds.map(parse_example)
val_parse_ds=val_ds.map(parse_example)
test_parse_ds=test_ds.map(parse_example)


In [0]:
def clean_string(features):
    revised = tf.strings.regex_replace(features['sentence'], "\.\.\.", "", replace_global=True)
    revised = tf.strings.regex_replace(revised, "\\'", "'", replace_global=True)
    revised = tf.strings.regex_replace(revised, "\\n", "", replace_global=True)
    features["sentence"]=revised
    return features

train_clean_ds = train_parse_ds.map(lambda features: clean_string(features))
val_clean_ds = val_parse_ds.map(lambda features: clean_string(features))
test_clean_ds =  test_parse_ds.map(lambda features: clean_string(features))    


In [14]:
NUM_LABELS=2

config=XLNetConfig.from_pretrained("xlnet-base-cased", num_labels=NUM_LABELS, dropout=0.2)
tokenizer=XLNetTokenizer.from_pretrained("xlnet-base-cased")
model= TFXLNetForSequenceClassification.from_pretrained("xlnet-base-cased",config=config)

In [15]:
config

XLNetConfig {
  "architectures": [
    "XLNetLMHeadModel"
  ],
  "attn_type": "bi",
  "bi_data": false,
  "bos_token_id": 1,
  "clamp_len": -1,
  "d_head": 64,
  "d_inner": 3072,
  "d_model": 768,
  "dropout": 0.2,
  "end_n_top": 5,
  "eos_token_id": 2,
  "ff_activation": "gelu",
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-12,
  "mem_len": null,
  "model_type": "xlnet",
  "n_head": 12,
  "n_layer": 12,
  "pad_token_id": 5,
  "reuse_len": null,
  "same_length": false,
  "start_n_top": 5,
  "summary_activation": "tanh",
  "summary_last_dropout": 0.1,
  "summary_type": "last",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 250
    }
  },
  "untie_r": true,
  "vocab_size": 32000
}

In [0]:
train_dataset = glue_convert_examples_to_features(examples=train_clean_ds, tokenizer=tokenizer
                                                  , max_length=512, task='sst-2',
                                                  label_list=['0','1']
                                                  )


In [0]:
val_dataset = glue_convert_examples_to_features(examples=val_clean_ds, tokenizer=tokenizer
                                                  , max_length=512, task='sst-2',
                                                  label_list=['0','1']
                                                  )

In [0]:
BATCH_SIZE= 6
EVAL_BATCH_SIZE= 12

train_dataset= train_dataset.shuffle(n_Tr).batch(BATCH_SIZE).repeat(-1)

val_dataset=val_dataset.batch(EVAL_BATCH_SIZE)


In [0]:
optimizer=tf.keras.optimizers.Adam(lr=2e-5,epsilon=1e-8)
loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric=tf.keras.metrics.SparseCategoricalAccuracy("accuracy")

model.compile(optimizer=optimizer,loss=loss,metrics=[metric])

In [0]:
train_steps= n_Tr // BATCH_SIZE
val_steps= n_Val // EVAL_BATCH_SIZE

In [21]:
model.summary()

Model: "tfxl_net_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
transformer (TFXLNetMainLaye multiple                  116718336 
_________________________________________________________________
sequence_summary (TFSequence multiple                  590592    
_________________________________________________________________
logits_proj (Dense)          multiple                  1538      
Total params: 117,310,466
Trainable params: 117,310,466
Non-trainable params: 0
_________________________________________________________________


In [22]:
history= model.fit(train_dataset, epochs=1, validation_data=val_dataset, verbose=2, steps_per_epoch=train_steps, validation_steps= val_steps)

666/666 - 2145s - loss: 0.3095 - accuracy: 0.8724 - val_loss: 0.2173 - val_accuracy: 0.9217


In [0]:
test_dataset = glue_convert_examples_to_features(examples=test_clean_ds, tokenizer=tokenizer
                                                  , max_length=512, task='sst-2'
                                                  , label_list =['0', '1'])

In [0]:
test_dataset = test_dataset.batch(EVAL_BATCH_SIZE)

In [26]:
history1=model.evaluate(test_dataset)

417/417 [==============================] - 826s 2s/step - loss: 0.2080 - accuracy: 0.9208


In [27]:
tf.saved_model.save(model, 'XLNet-SST-2')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: XLNet-SST-2/assets
